In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
import shutil
from string import digits
import ast

from collections import Counter

from sklearn.decomposition import PCA, SparsePCA, TruncatedSVD, IncrementalPCA
from scipy.sparse import csr_matrix, lil_matrix
import scipy
from itertools import chain
import statistics
from sklearn.pipeline import make_pipeline
from sklearn import svm, preprocessing
from sklearn.preprocessing import StandardScaler, normalize, LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from numpy import expand_dims

from tensorflow.keras.utils import to_categorical
from tensorflow import concat
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import random
import tensorflow
import os
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras import layers, Input
from tensorflow.keras.layers import Rescaling, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, GlobalAveragePooling2D, GlobalMaxPooling2D, BatchNormalization
from tensorflow.keras import applications
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
import keras_tuner as kt
from keras.applications.vgg19 import preprocess_input
from tensorflow.image import rgb_to_grayscale, grayscale_to_rgb
from tensorflow import tile
from tqdm import tqdm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
image_size = 224
input_shape = (image_size, image_size, 3)

### Data table

In [2]:
# importation des données

categories = os.listdir('donnees_nettoyees/')
drawings = []
labels = list()
labels_str = list()

for folder in tqdm(categories):
    tmp_repertory = 'donnees_nettoyees/' + folder
    #print(folder)
    for img in os.listdir(tmp_repertory):
        if not img.startswith('.'):
            labels.append(folder)
            labels_str.append(img)
            img = load_img(tmp_repertory + '/' + img, target_size=(224,224,3))
            img = img_to_array(img)
            img = expand_dims(img,axis=0)
            img = preprocess_input(img)
            img = np.array(img)
            img = np.reshape(img, (224,224,3))
            drawings.append(img)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:12<00:00,  1.27s/it]


In [3]:
categories_humains = ['Crèche','PS','MS','GS','CE1','CE2','CM1','CM2','NOV','EXP']

labels_final = []
drawings_new = []
drawers_str = []
drawers_names = []

for i in range(len(labels)):
    if labels[i] in categories_humains:
        drawings_new.append(drawings[i])
        labels_final.append(labels[i])
        drawers_str.append(labels_str[i])

In [12]:
conditions = []
drawers_names = []

for k in range(len(labels_final)):
    if labels_final[k] != 'Crèche':
        drawers_names.append(drawers_str[k].split('-')[0] + '-' + drawers_str[k].split('-')[1])
        conditions.append(drawers_str[k].split('-')[2].split('.')[0])
    else:
        drawers_names.append(''.join([i for i in drawers_str[k] if not i.isdigit()]))
        conditions.append('LIBRE')
        #conditions.append(drawers_str[k].split('-')[2])
    #print(labels_final[k] + ' - ' +  drawers_names[k])
    
df_drawings = pd.DataFrame({ 'names' : drawers_str, 'labels' : labels_final, 'conditions' : conditions})
df_drawings['labels_conditions'] = df_drawings['labels'] + ' ' + df_drawings['conditions']
dict_data = Counter(df_drawings['labels_conditions'])

df_data = {'age' : [], 'nb_obs' : [], 'condition' : []}
for key, nb_obs in zip(dict_data.keys(), dict_data.values()):
    df_data['age'].append(key.split(' ')[0])
    df_data['condition'].append(key.split(' ')[1])
    df_data['nb_obs'].append(nb_obs)
df_data = pd.DataFrame(df_data)

df_data['age'] = pd.Categorical(
    df_data['age'], 
    categories=categories_humains, 
    ordered=True
)
df_data = df_data.sort_values('age')

In [15]:
list(np.unique(drawers_names))

['ADE-BAS',
 'ADR-HEM',
 'ADR-WEB',
 'AGA-AEC',
 'AGA-SIL',
 'AGN-SAU',
 'ALA-BEY',
 'ALB-DAN',
 'ALE-BON',
 'ALE-EMP',
 'ALE-OZT',
 'ALI-POR',
 'ANA-ANJ',
 'ANA-BAL',
 'ANA-BOL',
 'ANA-DUR',
 'ANA-GRI',
 'ANG-SAC',
 'ANN-BAR',
 'ANN-KAR',
 'ANN-NAL',
 'ANT-DEC',
 'ANT-FRO',
 'ANT-MAI',
 'APO-ROU',
 'APR-DAS',
 'ARM-FRO',
 'ARN-LEC',
 'ART-BAL',
 'ART-TAZ',
 'AUG-MON',
 'AUX-CEN',
 'AVA-SAN',
 'AYA-YIL',
 'AZR-YIL',
 'BAU-MIQ',
 'BEN-LET',
 'BLA-DEA',
 'CAM-FRO',
 'CAP-PIN',
 'CAR-HAB',
 'CAR-MEJ',
 'CEL-GAU',
 'CHL-DEC',
 'CHR-CAR',
 'CLA-HAH',
 'CLE-FER',
 'CLE-GAU',
 'CLE-MUL',
 'COL-AUB',
 'COM-TUL',
 'Cécile.JPG',
 'DAN-KAR',
 'DAV-SAI',
 'DEL-VAU',
 'ELE-BOL',
 'ELI-MAR',
 'ELY-MOU',
 'EMY-ESC',
 'EST-NAV',
 'ETI-FRA',
 'EUG-SAN',
 'EVA-KRU',
 'EVA-LAH',
 'EVA-SON',
 'FAN-CHE',
 'FAN-HEI',
 'FLO-CHA',
 'FLO-KLE',
 'FLO-SIE',
 'FRA-CRI',
 'GAB-AND',
 'GAB-BRE',
 'GAB-HEI',
 'GAB-MEZ',
 'GAR-LET',
 'GAU-ANN',
 'GER-DEV',
 'GRI-MES',
 'HIP-ANJ',
 'ISA-DEB',
 'ISA-JAC',
 'ISA-OLA',
 

### Mean scores of RGB models

In [16]:
enfants = ['Crèche','PS','MS','GS','CE1','CE2','CM1','CM2']
mean_acc = []
scores_dict = {}
scores_list = []
labels_RGB = []
for i in range(len(enfants)-1):
    enfants_groupe = enfants[i:i+2]
    with open('donnees_nettoyees_models/RGB/models_nettoyees/' + enfants_groupe[0] + enfants_groupe[1] +'/10_best_models_acc.txt') as f:
        lines = f.readlines()[0]
    tmp_acc = ast.literal_eval(lines)
    scores_dict[enfants_groupe[0] + enfants_groupe[1]] = tmp_acc
    scores_list.append(tmp_acc)

scores_list = list(chain(*scores_list))
for key in scores_dict.keys():
    print('For the models classifying adults VS ' + key + ', the mean accuracy is : ' + str(np.mean(scores_dict[key])))

For the models classifying adults VS CrèchePS, the mean accuracy is : 0.8838709533214569
For the models classifying adults VS PSMS, the mean accuracy is : 0.7727272868156433
For the models classifying adults VS MSGS, the mean accuracy is : 0.6909090876579285
For the models classifying adults VS GSCE1, the mean accuracy is : 0.6529411852359772
For the models classifying adults VS CE1CE2, the mean accuracy is : 0.5636363685131073
For the models classifying adults VS CE2CM1, the mean accuracy is : 0.65625
For the models classifying adults VS CM1CM2, the mean accuracy is : 0.6117647051811218


### Mean scores of grayscale models

In [8]:
enfants = ['Crèche','PS','MS','GS','CE1','CE2','CM1','CM2']
mean_acc = []
scores_dict = {}
scores_list = []
labels_BW = []
for i in range(len(enfants)-1):
    enfants_groupe = enfants[i:i+2]
    with open('donnees_nettoyees_models/BW/models_nettoyees_BW/' + enfants_groupe[0] + enfants_groupe[1] +'/10_best_models_acc.txt') as f:
        lines = f.readlines()[0]
    tmp_acc = ast.literal_eval(lines)
    scores_dict[enfants_groupe[0] + enfants_groupe[1]] = tmp_acc
    scores_list.append(tmp_acc)

scores_list = list(chain(*scores_list))
for key in scores_dict.keys():
    print('For the models classifying adults VS ' + key + ', the mean accuracy is : ' + str(np.mean(scores_dict[key])))

For the models classifying adults VS CrèchePS, the mean accuracy is : 0.9354838550090789
For the models classifying adults VS PSMS, the mean accuracy is : 0.639393949508667
For the models classifying adults VS MSGS, the mean accuracy is : 0.7848484933376312
For the models classifying adults VS GSCE1, the mean accuracy is : 0.7088235318660736
For the models classifying adults VS CE1CE2, the mean accuracy is : 0.6303030341863632
For the models classifying adults VS CE2CM1, the mean accuracy is : 0.68125
For the models classifying adults VS CM1CM2, the mean accuracy is : 0.679411768913269


### Best hyperparameters of RGB models

In [21]:
groups = {}
for models in ['CrèchePS', 'PSMS', 'MSGS', 'GSCE1', 'CE1CE2', 'CE2CM1', 'CM1CM2']:
    if not models.startswith('.'):
        with open('donnees_nettoyees_models/RGB/models_nettoyees/' + models + '/models_scores.txt') as f:
            lines = f.readlines()[1]
            lines = lines[:-1]
        with open('donnees_nettoyees_models/RGB/models_nettoyees/' + models + '/10_best_models_acc.txt') as f:
            acc = f.readlines()[0]
            acc = ast.literal_eval(acc)
            acc_mean = np.mean(acc)
            lines = lines + ', \'accuracy\': ' + str(acc_mean) + '}'
        groups[models] = ast.literal_eval(lines)
        


In [22]:
groups_reindex = pd.DataFrame(groups).reindex(['pool_value','dropout','BatchNorm','num_layers','units_0','units_1','units_2','lr','batch_size']).reset_index()

In [24]:
groups_reindex

,index,CrèchePS,PSMS,MSGS,GSCE1,CE1CE2,CE2CM1,CM1CM2
0,pool_value,None,avg,avg,None,avg,avg,avg
1,dropout,0.4,0.4,0.0,0.4,0.4,0.0,0.4
2,BatchNorm,1,1,1,1,0,0,0
3,num_layers,3,1,1,1,1,3,1
4,units_0,288,32,512,32,512,512,512
5,units_1,512,32,512,32,32,512,512
6,units_2,320,32,512,32,512,32,512
7,lr,0.1,0.1,0.1,0.1,0.1,0.1,0.1
8,batch_size,64,16,64,64,16,64,64


### Best hyperparameters of BW models

In [4]:
groups_BW = {}
for models in ['CrèchePS', 'PSMS', 'MSGS', 'GSCE1', 'CE1CE2', 'CE2CM1', 'CM1CM2']:
    if not models.startswith('.'):
        with open('donnees_nettoyees_models/BW/models_nettoyees_BW/' + models + '/models_scores.txt') as f:
            lines = f.readlines()[1]
            lines = lines[:-1]
        with open('donnees_nettoyees_models/BW/models_nettoyees_BW/' + models + '/10_best_models_acc.txt') as f:
            acc = f.readlines()[0]
            acc = ast.literal_eval(acc)
            acc_mean = np.mean(acc)
            lines = lines + ', \'accuracy\': ' + str(acc_mean) + '}'
        groups_BW[models] = ast.literal_eval(lines)
        


In [5]:
groups_BW_reindex = pd.DataFrame(groups_BW).reindex(['pool_value','dropout','BatchNorm','num_layers','units_0','units_1','units_2','lr','batch_size']).reset_index()

In [6]:
groups_BW_reindex

,index,CrèchePS,PSMS,MSGS,GSCE1,CE1CE2,CE2CM1,CM1CM2
0,pool_value,avg,avg,avg,avg,avg,avg,avg
1,dropout,0.0,0.0,0.0,0.4,0.4,0.0,0.4
2,BatchNorm,0,0,1,1,1,0,1
3,num_layers,1,3,1,1,3,3,1
4,units_0,512,32,448,32,96,32,32
5,units_1,384,512,32,32,512,512,256
6,units_2,256,512,288,512,448,512,32
7,lr,0.1,0.1,0.1,0.1,0.1,0.1,0.1
8,batch_size,64,64,16,16,64,64,32
